In [1]:
import pandas as pd
import json

from utils import postRequest, putRequest
from config import getBaseURL
from updateCampaign import update

In [7]:
def toJSON(df):
    df.to_json("testCampaign.json",
           orient = "records",
           date_format = "epoch",
           double_precision = 10,
           force_ascii = True,
           date_unit = "ms",
           default_handler = None)

In [8]:
def formatCSV(df):
    drop = ['Campaign Group', 'Campaign Type']
    # Not seeing Campaign Group in the API
    # Rick has Campaign Type as a single value but API has key-value pair
    # for Frequency Capping you must specify both "how many times" and "hours"
    # Rick has OBA Provider as a single value but API has key-value pair
    # Could not get oba_provider working as a dictionary "invalid key", only as a 
    #    scalar (oba_provider_id)
    # Are you ever going to have Retail Markups?
    # Max Bid can't have $ in it, numeric only
    # Not seeing Budget Type in the API doc
    # Not seeing Budget Spend in the API doc; Total Spend is there
    # What is the difference between col G and O? Both look like impression caps
    # I am not seeing "Monthly Impression Cap" col P
    # Auto Optimize Goal should be True/False, although it looked like it worked with "enabled"
    # Not seeing anything for Location Type; maybe just a screen field?
    # geo-Fences: TBD
    # Geo Fence Recency: get codes
    35, 37
    
    
    
    keep = ['Campaign Name', 'Custom ID', 'Start Date', 'End Date', 'Impressions',
            'Per hours', 'Comments', 'max_bid', 'goal_type', 'goal_value',
            'Auto Optimize Goal', ]
    df = df[keep]
    
    d = {'Campaign Name': 'name', 'Custom ID': 'custom_id',
        'Start Date': 'start_date', 'End Date': 'end_date',
        'Impressions': 'impression_cap', 'Per hours': 'hours',
         'Max Bid': 'max_bid', 'Auto Optimize Goal': 'auto_optimize',}
    df.rename(columns=d, inplace=True)
    return df

In [9]:
def formatPayload(df):
    d2={}
    for col in df.columns:
        if col == 'hours':
            d2['frequency_capping'] = {"how_many_times": '2', 'hours': str(df.loc[0,col])}
        elif col == 'oba_provider':
            pass
        elif col in ['Comments', 'goal_type', 'goal_value']:
            pass
        else:
            d2[col]= str(df.loc[0,col])
    
    d1={}
    d1['campaign'] = d2
    print(d1)
    return d1

In [10]:
# process any comments
def addComment(df):
    comment = df.loc[0, 'Comments']
    if comment is not None:
        base = getBaseURL()
        url = base+'campaigns/'+str(campaignID)+'/comments'
        payload = {"comment": {"text": "this is Ricks first comment"}}
        payload = json.dumps(payload)
        resp = postRequest(url, payload)

In [11]:
# Campaign Goals
def campaignGoals(df):
    goalType = df.loc[0, 'goal_type']
    goalAmt = df.loc[0, 'goal_value']
    base = getBaseURL()
    url = base+"organizations/"+str(orgID)+"/campaigns/"+str(campaignID)
    payload = {"campaign": {"campaign_goal": {"goal_type": goalType,
                                              'goal_value': goalAmt}
                           }
              }
    payload = json.dumps(payload)
    resp = putRequest(url, payload)
    return resp

In [ ]:
def updDeviceTypes():
    {
        "device_type_ids": [2, 3, 36, 37]
      }' \
  "https://app.simpli.fi/api/campaigns/697/device_types"

In [12]:
df = pd.read_csv('/home/tbrownex/data/SteveRoss/testCampaign.csv', skiprows=1)

In [13]:
# Spreadsheet should have these columns
df['max_bid']=3.5
df['goal_type'] = 'CPA'
df['goal_value'] = 2.0

In [14]:
df = formatCSV(df)
payload = formatPayload(df)
payload = json.dumps(payload)

{'campaign': {'name': 'Copy 1-13 Backup of Pizza Hut', 'custom_id': '123456789', 'start_date': '2022-01-13', 'end_date': '2022-05-15', 'impression_cap': '4', 'frequency_capping': {'how_many_times': '2', 'hours': '24'}, 'max_bid': '3.5', 'auto_optimize': 'Enabled ', 'geo_fencing_recency_id': 'Within 1 month'}}


/home/tbrownex/tensorflow/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [15]:
orgID = 352328
campaignID = 2893122
tom=update(orgID, campaignID, payload)

success


In [16]:
tom

{'campaigns': [{'resource': 'https://app.simpli.fi/api/organizations/266374/campaigns/2893122',
   'id': 2893122,
   'name': 'Copy 1-13 Backup of Pizza Hut',
   'custom_id': '123456789',
   'current_win_rate': 0.0,
   'daily_budget': 143.0,
   'auto_adjust_daily_budget': False,
   'monthly_budget': None,
   'total_budget': None,
   'status': 'Draft',
   'impression_cap': 4,
   'daily_impression_cap': None,
   'monthly_impression_cap': None,
   'auto_adjust_daily_impression_cap': False,
   'pacing': 100.0,
   'automated_pacing_enabled': True,
   'media_type_id': None,
   'auto_optimize': True,
   'click_attribution_window': 30,
   'view_attribution_window': 30,
   'start_date': '2022-01-13',
   'end_date': '2022-05-15',
   'bid': 3.5,
   'bid_type': {'id': 1, 'name': 'CPM'},
   'viewability': 'Not Set',
   'week_dayparting': [[0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     1

In [12]:
# Comment processing
comment = addComment(df)

In [19]:
campaignGoals(df)

https://app.simpli.fi/api/organizations/352328/campaigns/2893122
{"campaign": {"campaign_goal": {"goal_type": "CPA", "goal_value": 2.0}}}
success
